In [1]:
from credentials_local import user, password, rds_host
import pandas as pd
from boston_functions import *
from liquid import liquids
from garnish import garnishes
from fractions import Fraction
import pymysql
import random
import re

data = pd.read_csv("./mr-boston-all-glasses.csv")

data = data[data.loc[:, "glass-size"].notna()]

valid_units = ["oz", "tsp", "splash", "dash"]

fill_liquid = ["ginger ale", "carbonated water", "cola", "water", "chilled champagne", "soda water", 
               "club soda", "ginger ale or soda water", "lemon-lime soda", "ginger beer", "bitter lemon soda",
               "apple juice", "orange juice"]

invaild_ingredients = ['chopped', 'cut in half', 'cut in halves', 'cut into halves', 'flamed', 'hulled', 'long', 'skinned',
                       'whipped', "preferably b.a. reynold's", 'preferably jamaican', 'preferably japanese', 
                       'preferably pedro ximenez', "such as bittermen's elemakule", 'such as demerara', 
                       'such as islay or skye', 'such as nasturtium']

ingredient_indicies = range(3, 14)

all_recipies, measures = get_cocktail_recipies(data, ingredient_indicies, liquids, garnishes, invaild_ingredients, valid_units)

In [2]:
#create glass list
glasses = []
for i in range(len(all_recipies)):
    glasses.append(all_recipies[i]["glass"])
glasses = list(set(glasses))

In [3]:
df = pd.DataFrame(all_recipies)
df.head()

,category,glass,glass_size,instructions,name,recipe
0,Cocktail Classics,Old-Fashioned Glass,6,Combine ingredients with a cup of crushed ice ...,Gauguin,"[[2 oz, batavia arrack or light rum], [1 oz, p..."
1,Cocktail Classics,Old-Fashioned Glass,6,Shake with ice and strain into old-fashioned g...,Fort Lauderdale,"[[1 1/2 oz, batavia arrack or light rum], [1/2..."
2,Cordials and Liqueurs,Old-Fashioned Glass,6,Pour into ice-filled old-fashioned glass. Garn...,Apple Pie,"[[3 oz, apple schnapps], [1 oz, cinnamon schna..."
3,Cocktail Classics,Cocktail Glass,6,Shake with ice and strain into cocktail glass.,Cuban Cocktail No. 1,"[[1/2 oz, fresh lime juice], [add, powdered su..."
4,Cocktail Classics,Collins Glass,14,"Mix all ingredients except curacao with ice, s...",Cool Carlos,"[[1 1/2 oz, dark rum], [2 oz, each cranberry j..."


In [4]:
#create glass sizes list
glass_sizes = []
for glass in glasses:
    sizes = df.loc[df["glass"] == glass, "glass_size"]
    glass_sizes.append(list(sizes)[0])
    
#create glass size dictionary
glass_size_dict = {}
for i in range(len(glasses)):
    glass_size_dict[glasses[i]] = glass_sizes[i]
glass_size_dict

{'Sherry Glass': 4,
 'Hurricane Glass': 14,
 'Cordial or Pony Glass': 2,
 'Sour Glass': 5,
 'Highball Glass': 7,
 'Irish Coffee Glass': 8,
 'Cocktail Glass': 6,
 'Red-Wine Glass': 12,
 'Shot Glass': 1,
 'Beer Mug': 10,
 'Collins Glass': 14,
 'White-Wine Glass': 10,
 'Champagne Flute': 6,
 'Brandy Snifter': 8,
 'Pousse-Cafe Glass': 1,
 'Punch Cup': 3,
 'Old-Fashioned Glass': 6}

In [5]:
text_files = ["MACKs_AI_Classics_2.txt", "MACKs_AI_Classics_5.txt", "MACKs_AI_Classics_10.txt"]

In [6]:
def parse_ai_text(text_file):
    #to track how many records are dropped due to what criteria
    all_lines = 0
    good_cocktails = 0
    too_many_ingredients = 0
    wrong_glass = 0
    bad_instructions = 0
    bad_volume = 0
    
    #connect to sql
    conn = pymysql.connect(rds_host, user=user, password=password, connect_timeout=50)
    cursor = conn.cursor()
    cursor.execute('USE cocktailproject')
    
    #open text file to read
    with open(text_file, "r") as file:
        for line in file:
    #         print(line)
            #split line
            split_line = line.split(" - ")
            all_lines += 1

            #get name
            name = split_line[0]
            print(f"COCKTAIL NAME: {name}")

            #get glass
            glass = split_line[1]
            print(f"GLASS NAME: {glass}")

            #get instructions
            instructions = split_line[-1:][0][:-1]
            instructions = re.sub(r'["]', '\'', instructions)
            print(f"INSTRUCTIONS: {instructions}")


            garnish_list = []
            measure_list = []
            measure_float_list = []
            liquid_list = []
            total_volume = 0
            try:
                glass_volume = glass_size_dict[glass]
            except KeyError:
                glass_volume = 0


            #get number of ingredients (split length minus name, glass, instructions)
            ingredients_length = len(split_line) - 3
            #loop over ingredients
            for i in range(3, ingredients_length + 2):
                ingredient = split_line[i]

                #if ingredient begins with 'add', its a garnish
                if ingredient[:3] == "add":
                    garnish = ingredient.split("add ")[1]
                    print(f"GARNISH NAME: {garnish}")
                    garnish_list.append(garnish)

                #if ingredient starts with 'fill',
                elif split_line[i][:4] == "fill":
                    measure_list.append("fill")
                    print("MEASURE: fill")

                    #get liquid ingredient
                    liquid = split_line[i].split("fill ")[1]
                    liquid_list.append(liquid.strip())
                    print(f"LIQUID NAME: {liquid}")

                    #calculate measure_float
                    remaining_volume = float(glass_volume) - float(total_volume)
                    measure_float = float(remaining_volume / 2)
                    measure_float_list.append(measure_float)
                    print(f"MEASURE FLOAT: {measure_float}")

                    garnish_list.append(garnish)   


                else:
                    try:
                        #if ingredient begins with number, its a meausure and a liquid
                        int(ingredient[:1])
                    except ValueError:
                        pass
                    #split ingredient on spaces
                    split_ingredient = ingredient.split(" ")                

                    #loop over splits
                    for j in range(len(split_ingredient)):
                        #if split is unit of measure, check for compound fraction
                        if split_ingredient[j] in ["oz", "dash", "splash", "tsp"]:
                            units = split_ingredient[j].strip()                        

                            #if j-2 > -1, there is a compound fraction
                            if j-2 > -1:
                                #get 2 parts of compound fraction
                                fraction = split_ingredient[j-1].strip()
                                whole_number = split_ingredient[j-2].strip()                           

                                #recombine compound fraction
                                measure = whole_number+ " " +fraction
                                measure_list.append(measure + " " + units)
                                print(f"MEASURE: {measure + ' ' + units}")

                                #get liquid ingredient
                                liquid = ingredient.split(measure + " " + units)[1]
                                liquid_list.append(liquid.strip())                            
                                print(f"LIQUID: {liquid}")

                                #convert measure to float
                                try:
                                    measure_float = float(sum(Fraction(s) for s in measure.split()))
                                except ValueError:
                                    pass

                                #if unit is smaller than oz, convert measure to oz
                                if ingredient[:3] == "tsp":
                                    measure_float = measure_float / 6
                                elif ingredient[:4] == "dash":
                                    measure_float = measure_float / 32
                                elif ingredient[:6] == "splash":
                                    measure_float = measure_float / 5
                                measure_float_list.append(measure_float)
                                print(f"MEASURE FLOAT: {measure_float}")

                                #add measure to total volume
                                total_volume += measure_float

                            else: #just a single number or fraction
                                #get measure
                                measure = split_ingredient[j-1].strip()
                                measure_list.append(measure + " " + units)
                                print(f"MEASURE: {measure + ' ' + units}")


                                #get liquid ingredient
                                liquid = ingredient.split(measure + " " + units)[1]
                                liquid_list.append(liquid.strip())
                                print(f"LIQUID: {liquid}")

                                #convert to float
                                measure_float = float(sum(Fraction(s) for s in measure.split()))
                                #if unit is smaller than oz, convert measure to oz
                                if ingredient[:3] == "tsp":
                                    measure_float = measure_float / 6
                                elif ingredient[:4] == "dash":
                                    measure_float = measure_float / 32
                                elif ingredient[:6] == "splash":
                                    measure_float = measure_float / 5
                                measure_float_list.append(measure_float)
                                print(f"MEASURE FLOAT: {measure_float}")

                                #add measure to total volume    
                                total_volume += measure_float

            #if more than 6 ingredients, skip
            if len(split_line) > 10:
#                 print("INGREDIENTS ERRORRRRRRRRRRR")
                too_many_ingredients += 1
                pass

            elif glass not in glasses:
#                 print("GLASS ERRORRRRRRRRRR")
                wrong_glass += 1
                pass

            elif instructions[:3] == "add":
#                 print("INSTRUCTIONS ERRORRRRRRRRR")
                bad_instructions += 1
                pass 

            elif total_volume > glass_volume:
#                 print("VOLUME ERRORRRRRRRRRRRR")
                bad_volume += 1

            else:
                good_cocktails += 1
                
                #insert cocktail name and instructions into cocktail table
                sql = f'INSERT INTO Cocktails (Cocktail_Name, Instructions) VALUES ("{name}", "{instructions}");'
                cursor.execute(sql)
                conn.commit()
                print("Cocktail Table Updated")
                
                for liquid in liquid_list:
                    if liquid not in liquids:
                        #generate random hex color
                        hex_color = "#{:06x}".format(random.randint(0, 0xFFFFFF))
                        #insert new liquid and color into liquids table
                        sql = f'INSERT INTO Liquids (Liquid_Name, Color) VALUES ("{liquid}", "{hex_color}");'
                        cursor.execute(sql)
                        conn.commit()
                        print("New Liquid Added to Liquids Table")
                        
                for garnish in garnish_list:
                    if garnish not in garnishes:
                        #insert new garnish into garnishes table
                        sql = f'INSERT INTO Garnishes (Garnish_Name) VALUES ("{garnish}");'
                        cursor.execute(sql)
                        conn.commit()
                        print("New Garnish Added to Garnishes Table")
                        
                #find category id
                sql = "SELECT Category_ID FROM Categories WHERE Category_Name='AI Instant Classic';"
                cursor.execute(sql)
                data = cursor.fetchall()
                category_id = data[0][0]
                
                #find glass id
                sql = f"SELECT Glass_ID FROM Glasses WHERE Glass_Name='{glass}';"
                cursor.execute(sql)
                data = cursor.fetchall()
                glass_id = data[0][0]
                
                #find cocktail id (last row in table)
                sql = f"SELECT Cocktail_ID FROM Cocktails ORDER BY Cocktail_ID DESC LIMIT 1;"
                cursor.execute(sql)
                data = cursor.fetchall()
                cocktail_id = data[0][0]
                
                #update cocktail table entry with category id and glass id
                sql = f"UPDATE Cocktails SET Category_ID='{category_id}', Glass_ID='{glass_id}' WHERE Cocktail_ID='{cocktail_id}';"
                cursor.execute(sql)
                conn.commit()
                print("Cocktail Table Updated Again")
                
                for i in range(len(liquid_list)):
                    liquid = liquid_list[i]
                    measure = measure_list[i]
                    measure_float = measure_float_list[i]
                    
                    #find liquid id
                    sql = f'SELECT Liquid_ID FROM Liquids WHERE Liquid_Name="{liquid}";'
                    cursor.execute(sql)
                    data = cursor.fetchall()
                    
                    try:
                        liquid_id = data[0][0]
                    except IndexError:
                        pass
                    
                    #insert entry into liquid instructions table
                    sql = f"INSERT INTO Liquid_Instructions (Cocktail_ID, Liquid_ID, Measure, Measure_Float) VALUES ('{cocktail_id}', '{liquid_id}', '{measure}', '{measure_float}');"
                    cursor.execute(sql)
                    conn.commit()
                    print("Liquid Instructions Table Updated")
                    
                for garnish in garnish_list:
                    sql = f"SELECT Garnish_ID FROM Garnishes WHERE Garnish_Name='{garnish}';"
                    cursor.execute(sql)
                    data = cursor.fetchall()
                    
                    garnish_id = data[0][0]
                    
                    #insert entry into garnish instructions table
                    sql = f"INSERT INTO Garnish_Instructions (Cocktail_ID, Garnish_ID) VALUES ('{cocktail_id}', '{garnish_id}');"
                    cursor.execute(sql)
                    conn.commit()
                    print("Garnish Instructions Table Updated")
    
            print("----------------------")
    
    #close sql connection
    conn.close()
    print(all_lines, too_many_ingredients, wrong_glass, bad_instructions, bad_volume, good_cocktails)
    print("####################################")

In [7]:
for file in text_files:
    parse_ai_text(file)

COCKTAIL NAME: Black Hawk
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake with ice and strain into chilled cocktail glass. Add
MEASURE: 1 oz
LIQUID:  fresh lemon juice
MEASURE FLOAT: 1.0
MEASURE: 1/2 oz
LIQUID:  simple syrup
MEASURE FLOAT: 0.5
MEASURE: 1 oz
LIQUID:  ginger ale or soda water
MEASURE FLOAT: 1.0
GARNISH NAME: orange and lemon wheels
GARNISH NAME: pineapple wedge skewered with maraschino cherry
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Garnish Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: Biffy Cocktail
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake first four ingredients with ice and strain into ice-filled Collins glass. Garnis
MEASURE: 1 1/2 oz
LIQUID:  apricot flavored brandy
MEASURE FLOAT: 1.5
MEASURE: 1 oz
LIQUID:  batavia arrack or light rum
MEASURE FLOAT: 1.0
MEASURE: 1/2 oz
LIQUID:  creme de banana
MEAS

Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
----------------------
COCKTAIL NAME: The Blinker
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Muddle 4 basil leaves in mixing glass. Add remaining ingredients. Shake with ice and double-strain into chilled cocktai
MEASURE: 1 1/2 oz
LIQUID:  blanco tequila
MEASURE FLOAT: 1.5
MEASURE: 1 oz
LIQUID:  fresh lime juice
MEASURE FLOAT: 1.0
MEASURE: 1/2 oz
LIQUID:  simple syrup
MEASURE FLOAT: 0.5
MEASURE: 1 oz
LIQUID:  passion fruit juice
MEASURE FLOAT: 1.0
GARNISH NAME: fresh basil leaf
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: Bourbon Cooler
GLASS NAME: Old-Fashioned Glass
INSTRUCTIONS: Shake with ice and strain into cocktail glass.
MEASURE: 1/2 oz
LIQUID:  anisette
MEASURE FLOAT: 0.5


Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: Brandy Crusta Cocktail
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Stir with ice and strain into chilled cocktail glass. Add lemon twist.
MEASURE: 1 oz
LIQUID:  triple sec
MEASURE FLOAT: 1.0
MEASURE: 1/2 oz
LIQUID:  absinthe substitute
MEASURE FLOAT: 0.5
MEASURE: 1/4 oz
LIQUID:  maraschino liqueur
MEASURE FLOAT: 0.25
MEASURE: 1 oz
LIQUID:  angostura bitters
MEASURE FLOAT: 1.0
GARNISH NAME: lemon twiste
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: Bronx Golden Cocktail
GLASS NAME: Sour Glass
INSTRUCTIONS: Shake with ice and strain into cocktail glass.
MEASURE: 1/2 oz
LIQUID:  d

Liquid Instructions Table Updated
----------------------
COCKTAIL NAME: Palmetto Cocktail
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Muddle cucumber, sage, and lime juice in mixing glass. Add tequila and syrup with ice and shake. Double-strain into ice-filled old-fashioned glass. Add lemon
MEASURE: 1/2 oz
LIQUID:  simple syrup
MEASURE FLOAT: 0.5
MEASURE: 1 oz
LIQUID:  ginger ale or soda water
MEASURE FLOAT: 1.0
GARNISH NAME: strawberry slice
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: Bourbon Cooler
GLASS NAME: Old-Fashioned Glass
INSTRUCTIONS: Shake with ice and strain into chilled cocktail glass.
MEASURE: 1 oz
LIQUID:  fresh lemon juice
MEASURE FLOAT: 1.0
MEASURE: 1 oz
LIQUID:  grenadine
MEASURE FLOAT: 1.0
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
---------------

Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: Black Hawk
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake with ice and strain into cocktail glass.
MEASURE: 1 oz
LIQUID:  fresh lemon juice
MEASURE FLOAT: 1.0
MEASURE: 1 oz
LIQUID:  grenadine
MEASURE FLOAT: 1.0
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
----------------------
COCKTAIL NAME: The Blinker
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake with ice and strain into ice-filled old-fashioned glass. Garnish wi
MEASURE: 1 1/2 oz
LIQUID:  blanco tequila
MEASURE FLOAT: 1.5
MEASURE: 1/2 oz
LIQUID:  fresh lime juice
MEASURE FLOAT: 0.5
MEASURE: fill
LIQUID NAME: ginger ale or soda water
MEASURE FLOAT: 2.0
GARNISH NAME: orange and lemon wheels
GARNISH NAME

...

Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: Trinity Cocktail
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake with ice and strain into chilled cocktail glass.
MEASURE: 1/2 oz
LIQUID:  green creme de menthe
MEASURE FLOAT: 0.5
MEASURE: 1 oz
LIQUID:  fresh lemon juice
MEASURE FLOAT: 1.0
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
----------------------
COCKTAIL NAME: Green Opal
GLASS NAME: Red-Wine Glass
INSTRUCTIONS: Combine ingredients and a half-cup of crushed ice in an electric blender. Blend at low speed and pour into champagne flute.
MEASURE: 3 oz
LIQUID:  coconut milk
MEASURE FLOAT: 3.0
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
----------------------
COCKTAIL NAME: Red Swizzle
GLASS NAME: Collins Glass
INSTRUCTIONS: Shake rum, pineapple 

Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: Chi-Town Flip
GLASS NAME: Collins Glass
INSTRUCTIONS: Shake with ice and strain into chilled cocktail glass.
MEASURE: 1 1/2 oz
LIQUID:  rose's lime juice
MEASURE FLOAT: 1.5
MEASURE: 1 1/2 oz
LIQUID:  maple syrup
MEASURE FLOAT: 1.5
MEASURE: 1 oz
LIQUID:  angostura bitters
MEASURE FLOAT: 1.0
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
----------------------
COCKTAIL NAME: Vodka Daisy
GLASS NAME: Beer Mug
INSTRUCTIONS: Shake and strain into ice-filled old-fashioned glass. Add orange twist.
MEASURE: 1 oz
LIQUID:  each orange juice and sweet sherry
MEASURE FLOAT: 1.0
MEASURE: 1 oz
LIQUID:  raspberry syrup
MEASURE FLOAT: 1.0
GARNISH NAME: small pineapple slice
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructi

Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
----------------------
COCKTAIL NAME: Thunder
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake with ice and strain into cocktail glass. Serve with a twist of lemon peel or olive, if desired.
GARNISH NAME: egg yolk
MEASURE: 1 1/2 oz
LIQUID:  cherry  flavored brandy
MEASURE FLOAT: 1.5
GARNISH NAME: egg white
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Garnish Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: The juilos
GLASS NAME: Collins Glass
INSTRUCTIONS: Put ice cubes in collins glass and add lime vodka. Fill with lemon and salt; reserve lemon. Shake remaining ingredients with ice in mixing glass. Pour with ice into collins glass and soda water or old-fashicA, dum and fill with 
MEASURE: fill
LIQUID NAME: lemon-lime soda
MEASURE FLOAT: 7.0
Cock

Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Garnish Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: Canadian Pineapple
GLASS NAME: Old-Fashioned Glass
INSTRUCTIONS: Shake and strain into ice-filled highball glass. Top with soda water. Garnish with strawberry and basil.
MEASURE: 1/2 oz
LIQUID:  sakd rum
MEASURE FLOAT: 0.5
MEASURE: 3/4 oz
LIQUID:  sweet vermouth
MEASURE FLOAT: 0.75
MEASURE: 3/4 oz
LIQUID:  benedictine
MEASURE FLOAT: 0.75
MEASURE: 1/2 oz
LIQUID:  white creme de cacao
MEASURE FLOAT: 0.5
MEASURE: 1/2 oz
LIQUID:  chile syrup
MEASURE FLOAT: 0.5
Cocktail Table Updated
New Liquid Added to Liquids Table
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
----------------------
COCKTAIL NAME: American Trilogy
GLASS NAME: Old-Fashion

Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
----------------------
COCKTAIL NAME: Spanish Town Cocktail
GLASS NAME: Cocktail Glass
INSTRUCTIONS: Shake with ice and strain into chilled cocktail glass.
MEASURE: 1 oz
LIQUID:  angostura bitters
MEASURE FLOAT: 1.0
GARNISH NAME: lemon twiste
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Garnish Instructions Table Updated
----------------------
COCKTAIL NAME: Royal Cocktail
GLASS NAME: Sour Glass
INSTRUCTIONS: Shake with ice and strain into cocktail glass. Float an unbroken egg yolk on top
GARNISH NAME: powdered sugar
MEASURE: 1 oz
LIQUID:  ginger ale or soda water
MEASURE FLOAT: 1.0
MEASURE: 2 oz
LIQUID:  lime vodka
MEASURE FLOAT: 2.0
MEASURE: 1 oz
LIQUID:  pineapple juice
MEASURE FLOAT: 1.0
Cocktail Table Updated
Cocktail Table Updated Again
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Liquid Instructions Table Updated
Garnish